# Factor GRM and REML Analysis

This notebook demonstrates how to:
1. Preprocess PLINK bfiles and compute Factor GRMs (heterozygous, homozygous, and interaction terms).
2. Perform REML analysis using GCTA.
3. Parse and save the REML results for further analysis.

Follow the steps below to reproduce the analysis.

---
# Step 0: Preprocess PLINK bfiles
---

Before proceeding with GRM computation and REML analysis, ensure that your PLINK bfiles are preprocessed.  
This includes quality control (QC), aligning to the major reference allele, and generating allele frequency files.

Refer to the `plink.ipynb` notebook for detailed instructions on preprocessing PLINK bfiles.  
The preprocessing pipeline in `plink.ipynb` performs the following steps:
1. **Quality Control (QC)**: Filters samples and variants based on criteria such as MAF, HWE, and missingness.
2. **Alignment**: Aligns the dataset to the major reference allele.
3. **Allele Frequency Calculation**: Computes allele frequencies for the dataset.

Once preprocessing is complete, the output files will be ready for GRM computation in this notebook.

---
# Step 1: Set up the environment
---

In [7]:
# Define the root path of the project and import necessary modules
import ipynbname
import os, sys

# Automatically detect the notebook's directory
notebook_path = ipynbname.path()
notebook_dir = os.path.dirname(notebook_path)
ROOT_PATH = os.path.abspath(os.path.join(notebook_dir, '..'))

# Add the project root to the Python path
sys.path.insert(0, os.path.join(ROOT_PATH))

print(f"Project root path: {ROOT_PATH}")

Project root path: /Users/jerry/Documents/JerryProject/1.Project/Factor/Analysis/src/pyGCTA


---
# Step 2: Compute GRMs
---

This step calculates GRMs (additive, dominance, and factor) from PLINK bfiles.

In [8]:
from gcta import GCTA
gcta = GCTA()

for standardization in ["add", "dom", "factor"]:
    print(f"Calculating GRM with standardization: {standardization}")
    
    # Compute GRM
    gcta.calculate_grm(
        bfile=f"{ROOT_PATH}/test/bfile/test.majref",
        standardization=standardization,
        afreq=f"{ROOT_PATH}/test/bfile/test.majref.afreq",
        maf_threshold=0.01,
    )
    
    # Save GRM
    gcta.save_grm(out_prefix=f"{ROOT_PATH}/test/grm/test.majref")
    print(f"GRM saved for {standardization} standardization.")
    
    # Remove GRM to save memory
    if standardization == "factor":
        for grm_type in ["het", "hom", "hethom"]:
            gcta.remove_grm(grm_type=grm_type)
    else:
        gcta.remove_grm(grm_type=standardization)

GCTA Python v0.1.0 initialized
Calculating GRM with standardization: add
> Load PLINK files
|   Loaded 3900 samples and 985 SNPs
> Load AFREQ file
|   Loaded 985 SNPs allele frequencies
|   984 SNPs after MAF thresholding
|   Using 984 common SNPs
> Calculate GRM
|   Loading BED file
|   Using 3900 samples and 984 SNPs
|   Successfully computed GRM!
|   Save: /Users/jerry/Documents/JerryProject/1.Project/Factor/Analysis/src/pyGCTA/test/grm/test.majref.add.grm.id
|   Save: /Users/jerry/Documents/JerryProject/1.Project/Factor/Analysis/src/pyGCTA/test/grm/test.majref.add.grm.bin
|   Save: /Users/jerry/Documents/JerryProject/1.Project/Factor/Analysis/src/pyGCTA/test/grm/test.majref.add.grm.N.bin
|   Successfully saved GRM!
GRM saved for add standardization.
Calculating GRM with standardization: dom
> Load PLINK files
|   Loaded 3900 samples and 985 SNPs
> Load AFREQ file
|   Loaded 985 SNPs allele frequencies
|   984 SNPs after MAF thresholding
|   Using 984 common SNPs
> Calculate GRM
|  

---
# Step 3: Perform REML Analysis
---
This step performs REML analysis using GCTA. Ensure the GCTA executable path is correct.


In [9]:
for enc_model in ["AD", "Factor"]:
    print(f"Running REML analysis for model: {enc_model}")
    
    GCTA_PATH = f"{ROOT_PATH}/gcta64/bin/gcta64"  # Update this path if necessary
    MGRM_FILE = f"{ROOT_PATH}/test/grm/{enc_model}.mgrm"
    PHENOTYPE_FILE = f"{ROOT_PATH}/test/test.pheno"
    OUTPUT_PREFIX = f"{ROOT_PATH}/test/reml/test.majref.{enc_model}"

    gcta.reml(
        gcta_path=GCTA_PATH,
        mgrm_file=MGRM_FILE,
        phenotype_file=PHENOTYPE_FILE,
        out_prefix=OUTPUT_PREFIX,
        no_contrain=True,
        no_lrt=True,
        n_threads=5
    )
    print(f"REML analysis completed for model: {enc_model}")

Running REML analysis for model: AD
*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version v1.95.0 Mac
* (C) 2010-present, Yang Lab, Westlake University
* Please report bugs to Jian Yang <jian.yang@westlake.edu.cn>
*******************************************************************
Analysis started at 16:56:04 KST on Tue Sep 09 2025.
Hostname: JerrysMacPro14.local

Accepted options:
--reml
--mgrm /Users/jerry/Documents/JerryProject/1.Project/Factor/Analysis/src/pyGCTA/test/grm/AD.mgrm
--pheno /Users/jerry/Documents/JerryProject/1.Project/Factor/Analysis/src/pyGCTA/test/test.pheno
--out /Users/jerry/Documents/JerryProject/1.Project/Factor/Analysis/src/pyGCTA/test/reml/test.majref.AD
--reml-no-constrain
--reml-no-lrt
--thread-num 5

Note: the program will be running on 5 threads.

Reading phenotypes from [/Users/jerry/Documents/JerryProject/1.Project/Factor/Analysis/src/pyGCTA/test/test.pheno].
Non-missing phenotypes of 3

---
# Step 4: Parse and Save REML Results
---
This step parses the REML results and saves them for further analysis.


In [10]:
FACTOR_HSQ_FILE = f"{ROOT_PATH}/test/reml/test.majref.FACTOR.hsq"
df_hsqF = gcta.load_factor_hsq(FACTOR_HSQ_FILE, save=True)

# Display the parsed results
print("Parsed REML Results:")
print(df_hsqF)

|   Must be (V1: het, V2: hethom, V3: hom)
metric        mean        SE
h2(het)   0.135376  0.012859
h2(hom)   0.518830  0.018762
h2(F)     0.654206  0.014317
rho       0.079623  0.117798
hom/het   3.832521  0.465894
Parsed REML Results:
None


---
## Summary
---

In this notebook, we:
1. Computed Factor GRMs from PLINK bfiles.
2. Performed REML analysis using GCTA.
3. Parsed and visualized the REML results.

You can now use the parsed results for further analysis or modeling. For more details, refer to the documentation.